In [1]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
from ir_measures import *

from config import MODELS

pd.set_option('display.width', 1000)

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
# Note, we now perform evaluation on the training set, so scores are inflated.
# We mostly do this as a sanity check to ensure that the models are working as expected.

dataset = pt.get_dataset('irds:ir-benchmarks/longeval-2023-01-20240423-training')

bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)
ltr_models = [
    pt.io.read_results(f'../runs/ows_ltr_{model}/ows_ltr_{model}.train_2024')
    for model in MODELS
]

pt.Experiment(
    [bm25] + ltr_models,
    dataset.get_topics(variant='query'),
    dataset.get_qrels(variant='relevance'),
    eval_metrics=[nDCG@10, nDCG, nDCG(judged_only=True)@10, nDCG(judged_only=True), MAP@10, MAP, MAP(judged_only=True)@10, MAP(judged_only=True)],
    names=['BM25'] + list(MODELS.keys()),
).round(3)

,name,nDCG@10,nDCG,nDCG(judged_only=True)@10,nDCG(judged_only=True),AP@10,AP,AP(judged_only=True)@10,AP(judged_only=True)
0,BM25,0.199,0.317,0.444,0.473,0.128,0.180,0.373,0.401
1,wows_only,0.523,0.550,0.587,0.591,0.437,0.457,0.516,0.527
2,wows_base_rerank,0.539,0.560,0.587,0.591,0.456,0.474,0.518,0.528
3,wows_all_rerank,0.537,0.561,0.585,0.590,0.444,0.468,0.512,0.524
4,wows_rerank_and_keyquery,0.555,0.570,0.591,0.594,0.479,0.494,0.524,0.534
5,wows_rerank_and_reverted_index,0.506,0.543,0.569,0.579,0.396,0.428,0.489,0.506
6,all,0.557,0.571,0.591,0.593,0.481,0.495,0.524,0.533
